## Test the MIRI MRS distortion solution (standalone and JWST pipeline versions) ##

### First import the things that we need from the pipeline code ###

In [1]:
import os as os
import numpy as np
from numpy.testing import utils
from numpy.testing import assert_allclose
import pdb as pdb


/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_31997/1106886355.py:3: DeprecationWarning: Importing from numpy.testing.utils is deprecated since 1.15.0, import from numpy.testing instead.
  from numpy.testing import utils


Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the local data directory environmental variable is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

### First we'll look at the standalone distortion solution (mrs_tools) ###

In [3]:
import miricoord.mrs.mrs_tools as mrst
mrst.set_toolversion('flt1')

Define a point on the MRS (0-indexed convention) and convert to v2,v3 for Ch1A

In [4]:
x=[194.0,231.0]
y=[512.0,700.0]
band='4C'

Convert this pixel location to the MRS local alpha,beta,lam coordinates

In [5]:

values=mrst.xytoabl(x,y,band)
v2,v3=mrst.abtov2v3(values['alpha'],values['beta'],band)

The returned values are a dictionary of different entries, including input pixels, output alpha/beta coordinates, wavelength (lam), and slice number/name.

In [6]:
values,v2,v3

({'x': array([194., 231.]),
  'y': array([512., 700.]),
  'alpha': array([-0.06619308, -0.01176626]),
  'beta': array([ 2.29611932, -1.64008523]),
  'lam': array([26.29237924, 25.35069458]),
  'slicenum': array([10,  4]),
  'slicename': array(['410C', '404C'], dtype='<U4')},
 array([-503.28627393, -502.59623788]),
 array([-321.91663845, -318.09196097]))

In [7]:
alpha,beta,lam=values['alpha'],values['beta'],values['lam']

In [8]:
alpha,beta,lam

(array([-0.06619308, -0.01176626]),
 array([ 2.29611932, -1.64008523]),
 array([26.29237924, 25.35069458]))

Note that not every pixel on the detector actually maps to alpha,beta,lam since many pixels are between slices.  Therefore we have a way to 'trim' the inputs so that the returned values are only for such valid pixels.  Let's define an input array of pixels in which some map to slices and some do not.

In [9]:
x,y=[50,62,70,80,90],[500,510,520,10,950]

In [10]:
values=mrst.xytoabl(x,y,'1A',trim=1)

Note now that while we sent in five coordinates, only four got returned as one was trimmed.  The output values['x'] array is the trimmed version of the input x coordinate.

In [11]:
values

{'x': array([50, 70, 80, 90]),
 'y': array([500, 520,  10, 950]),
 'alpha': array([-0.32902713, -1.05377755,  1.45131771, -1.39343472]),
 'beta': array([ 1.77210144, -0.17721014, -0.17721014,  1.59489129]),
 'lam': array([5.32581863, 5.35499686, 4.89288863, 5.69720936]),
 'slicenum': array([21, 10, 10, 20]),
 'slicename': array(['121A', '110A', '110A', '120A'], dtype='<U4')}

We can convert alpha,beta to v2,v3 as well:

In [12]:
v2,v3=mrst.abtov2v3(values['alpha'],values['beta'],'1A')

In [13]:
v2,v3

(array([-503.80056612, -504.3453711 , -501.86447088, -504.81762227]),
 array([-320.74815482, -318.6960657 , -319.06399426, -320.41515109]))

In [15]:
# Test the Ch1A boresight location.  In FLT-1 this is -503.3181535345708, -319.0277065586294
v2,v3=mrst.abtov2v3(0,0,'1A')
v2,v3

(-503.3181535345708, -319.0277065586294)

### Now let's test that the distortion transforms for all 12 channels/subbands match reference positions correctly: ###

In [16]:
mrst.testtransform()

Testing channel 1A
Testing channel 1B
Testing channel 1C
Testing channel 2A
Testing channel 2B
Testing channel 2C
Testing channel 3A
Testing channel 3B
Testing channel 3C
Testing channel 4A
Testing channel 4B
Testing channel 4C


### Now we'll look at the implementation of the distortion solution in the JWST pipeline (mrs_pipetools) ###

In [17]:
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

In [18]:
import miricoord.mrs.mrs_pipetools as mrspt
mrspt.set_toolversion('flt1')

Quick test of using them directly

In [19]:
x,y=486,115
mrspt.xytoabl(x,y,'1A')

(0.19426487464869524, -1.77210143797, 4.968998639243164)

Point to a SHORT exposure reduced through spec2:

In [20]:
file=data_dir+'testing/jw01029001001_04101_00001_mirifushort_flt1_cal.fits'

Read the datamodel from the file and print some information about it

In [21]:
im = datamodels.ImageModel(file)

In [22]:
print(im.meta.wcs)

   From       Transform   
---------- ---------------
  detector detector_to_abl
alpha_beta    abl_to_v2v3l
      v2v3   CompoundModel
v2v3vacorr   CompoundModel
     world            None


These are the available frames for the distortion transforms:

In [23]:
print(im.meta.wcs.available_frames)

['detector', 'alpha_beta', 'v2v3', 'v2v3vacorr', 'world']


Define a point on the MRS and check that the v2,v3 is correct

In [24]:
alpha,beta,lam=0.0,0.0,5.0

In [25]:
v2,v3,ltemp=im.meta.wcs.transform("alpha_beta", "v2v3", alpha,beta,lam)

In [26]:
print(v2,v3)

-503.3181535345708 -319.0277065586294


The V2,V3 location of the MRS 1A reference point should be -503.318, -319.028

We can check the RA/DEC too:

In [27]:
ra, dec, ltemp = im.meta.wcs.transform("v2v3", "world", v2,v3,lam)

In [28]:
print(ra,dec)

82.50317750454388 -69.97537777093329


### Now let's test that the distortion transforms for all 12 channels/subbands match reference positions correctly:  ###

In [29]:
mrspt.testtransform()

Testing channel 1A
Testing channel 1B
Testing channel 1C
Testing channel 2A
Testing channel 2B
Testing channel 2C
Testing channel 3A
Testing channel 3B
Testing channel 3C
Testing channel 4A
Testing channel 4B
Testing channel 4C
